# Create Folds

This notebook creates a stratified k-fold split of the dataset and saves the dataset with the folds as a new CSV file.

Grouped k-fold is used to prevent any overlap between the training and validation sets for the same patient.
Furthermore, each split is stratified based on the `expert_consensus` column to ensure a balanced distribution of classes across folds (or, as balanced as possible given the constraints of grouping by patient).

In [1]:
import sys
import os

if bool(os.environ.get("KAGGLE_URL_BASE", "")):
  import sys
  # running on kaggle
  sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
  # running locally
  sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import pandas as pd

from pathlib import Path
from src.utils.utils import get_raw_data_dir, get_processed_data_dir
from src.utils.k_folds_creator import KFoldCreator

2025-12-29 10:44:19,302 :: root :: INFO :: Initialising Utils
2025-12-29 10:44:20,162 :: root :: INFO :: Initialising Datasets
2025-12-29 10:44:20,607 :: root :: INFO :: Initialising Models
/home/david/miniconda3/envs/aicomp/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/david/miniconda3/envs/aicomp/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which ha

In [2]:
DATA_PATH = get_raw_data_dir()
PROCESSED_TRAIN_DATA_PATH = get_processed_data_dir() / "train_processed.csv"
OUTPUT_PATH = get_processed_data_dir() / "train_folds.csv"

SPLITS_N = 5
SEED = 42

In [4]:
train_df = pd.read_csv(PROCESSED_TRAIN_DATA_PATH)
train_df.head()

,eeg_id,spectrogram_id,min_offset,max_offset,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,568657,789577333,0.0,16.0,20654,Other,0.0,0.000000,0.25,0.000000,0.166667,0.583333
1,582999,1552638400,0.0,38.0,20230,LPD,0.0,0.857143,0.00,0.071429,0.000000,0.071429
2,642382,14960202,1008.0,1032.0,5955,Other,0.0,0.000000,0.00,0.000000,0.000000,1.000000
3,751790,618728447,908.0,908.0,38549,GPD,0.0,0.000000,1.00,0.000000,0.000000,0.000000
4,778705,52296320,0.0,0.0,40955,Other,0.0,0.000000,0.00,0.000000,0.000000,1.000000


In [5]:
fold_creator = KFoldCreator(n_splits=SPLITS_N, seed=SEED)
train_folds_df = fold_creator.create_folds(
    df=train_df, stratify_col="expert_consensus", group_col="patient_id"
)

train_folds_df.groupby(["fold", "expert_consensus"])[["eeg_id"]].count().T

fold                0                                 1                 ...  \
expert_consensus  GPD GRDA  LPD LRDA Other Seizure  GPD GRDA  LPD LRDA  ...   
eeg_id            636  385  377  168  1244     524  181  361  529  273  ...   

fold                3                       4                               
expert_consensus  LPD LRDA Other Seizure  GPD GRDA  LPD LRDA Other Seizure  
eeg_id            329   96  1260     430  251  345  542  229  1417     633  

[1 rows x 30 columns]

In [6]:
train_folds_df.to_csv(OUTPUT_PATH, index=False)
print(f"Train folds data saved to {OUTPUT_PATH}")

Train folds data saved to /home/david/git/aicomp/data/processed/train_folds.csv
